# Dataset Preparation for First ICU Prediction

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
np.set_printoptions(precision=2)

from utils.metrics import BinaryAvgMetrics
from utils.plots import *

import pandas as pd
import pickle
from pathlib import Path

In [3]:
from args import args
vars(args)

{'path': PosixPath('../data'),
 'workdir': PosixPath('../data/work_dir'),
 'dataset_csv': PosixPath('../data/processed_dataset.csv'),
 'temporal_pkl': PosixPath('../data/temporal_notes.pkl'),
 'min_freq': 3,
 'bc_threshold': {'lr': 0.47, 'rf': 0.32}}

## Functions

In [4]:
def get_metrics(cm, mu=1e-7):
  tn,fp,fn,tp = cm[0][0],cm[0][1],cm[1][0],cm[1][1]
  sensitivity = tp/(tp+fn+mu)
  specificity = tn/(tn+fp+mu)
  ppv = tp/(tp+fp+mu)
  npv = tn/(tn+fn+mu)

#   d = {
#     'sensitivity': np.round(sensitivity, 3),
#     'specificity': np.round(specificity, 3),
#     'ppv': np.round(ppv, 3),
#     'npv': np.round(npv, 3),
#   }

#   return pd.DataFrame(d.values(), index=d.keys(), columns=['Value'])
  return np.array([sensitivity, specificity, ppv, npv])

## 100 Run Performance Results

In [ ]:
models = list(args.bc_threshold.keys())

bams = {}

for model in models:
  with open(args.workdir/model/'preds.pkl', 'rb') as f:
    targs = pickle.load(f)
    preds = pickle.load(f)
    probs = pickle.load(f)
    
  bams[model] = BinaryAvgMetrics(targs, preds, probs)

all_metrics = {}

for key in bams.keys():
  all_metrics[key] = []
  for i in range(len(bams[key].get_avg_metrics())):
    all_metrics[key].append(bams[key].get_avg_metrics().iloc[i]['Value'])

metrics = pd.DataFrame(all_metrics, index=['sensitivity', 'specificity', 'ppv', 'auroc', 'npv', 'f1'])
metrics

In [ ]:
model = 'lr'
bams[model].get_avg_metrics()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_mean_roc(ax, bams[model].targs, bams[model].probs)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 6))

plot_confusion_matrix(ax[0], bams[model].cm_avg, classes=['not imminent', 'imminent'], normalize=False,\
                      title='Confusion Matrix Over Runs')
plot_confusion_matrix(ax[1], bams[model].cm_avg, classes=['not imminent', 'imminent'], normalize=True,\
                      title='Normalized Confusion Matrix Over Runs')
plt.show()

## Subsect data for Temporal Analysis

In [ ]:
df = pd.read_csv(args.dataset_csv, parse_dates=['admittime', 'dischtime', 'intime', 'charttime'])

df['relative_charttime'] = (df['charttime'] - df['intime']).apply(lambda x: int(x.value/(1e9*60*60)))
df['relative_charttime'].describe()

In [ ]:
# value in hours
e1 = -480
s1 = -120
e2 = -72
s2 = -24
step = 6

temporal_notes = []
period = []

for i in range(e1, s1, step):
  tmp = df.loc[((df['relative_charttime'] >= i) & (df['relative_charttime'] <= (i+step)))].copy()
  period.append(f'{i} ≤ t ≤ {i+step}')
  temporal_notes.append(tmp)

for i in range(e2, s2, step):
  tmp = df.loc[((df['relative_charttime'] >= i) & (df['relative_charttime'] <= (i+step)))].copy()
  period.append(f'{i} ≤ t ≤ {i+step}')
  temporal_notes.append(tmp)

These lengths should not be different...

In [ ]:
s = 0
for x in temporal_notes:
  s += len(x)
tmp = df.loc[((df['relative_charttime'] >= e1) & (df['relative_charttime'] <= s1))\
               | ((df['relative_charttime'] >= e2) & (df['relative_charttime'] <= s2))].copy()
print(s - len(tmp))

In [ ]:
with open(args.temporal_pkl, 'wb') as f:
  pickle.dump(temporal_notes, f)
  pickle.dump(period, f)

## Temporal Performance Analysis

In [5]:
with open(args.temporal_pkl, 'rb') as f:
  temporal_notes = pickle.load(f)
  period = pickle.load(f)

### Logistic Regression Model

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [7]:
with open(args.workdir/'lr'/'full_run.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)
  
vectorizer = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True,\
                              ngram_range=(2,2), vocabulary=vocabulary)  

In [8]:
metrics = np.zeros((len(period), 4))

In [11]:
import pdb

In [15]:
for i in range(len(period)):
  label = period[i]
  df = temporal_notes[i][['scispacy_note', 'class_label']]
  prob = clf.predict_proba(vectorizer.fit_transform(df['scispacy_note']))[:, 1]
  y_pred = (prob > args.bc_threshold['lr']).astype(np.int64)
  cm = confusion_matrix(df['class_label'].to_numpy(), y_pred)
  print(label)
  print(cmccc)
  pdb.set_trace()
  metrics[i] = get_metrics(cm)

-480 ≤ t ≤ -474
[[138  44]
 [  0   0]]
> <ipython-input-15-c97f846f7b9c>(10)<module>()
-> metrics[i] = get_metrics(cm)
(Pdb) c
-474 ≤ t ≤ -468
[[145  33]
 [  0   0]]
> <ipython-input-15-c97f846f7b9c>(9)<module>()
-> pdb.set_trace()
(Pdb) c
-468 ≤ t ≤ -462
[[153  45]
 [  0   0]]
> <ipython-input-15-c97f846f7b9c>(10)<module>()
-> metrics[i] = get_metrics(cm)
(Pdb) 
(Pdb) 
(Pdb) c
-462 ≤ t ≤ -456
[[178  37]
 [  0   0]]
> <ipython-input-15-c97f846f7b9c>(9)<module>()
-> pdb.set_trace()
(Pdb) c
-456 ≤ t ≤ -450
[[174  46]
 [  0   0]]
> <ipython-input-15-c97f846f7b9c>(10)<module>()
-> metrics[i] = get_metrics(cm)
(Pdb) c
-450 ≤ t ≤ -444
[[179  39]
 [  0   0]]
> <ipython-input-15-c97f846f7b9c>(9)<module>()
-> pdb.set_trace()
(Pdb) c
-444 ≤ t ≤ -438
[[205  50]
 [  0   0]]
> <ipython-input-15-c97f846f7b9c>(10)<module>()
-> metrics[i] = get_metrics(cm)
(Pdb) c
-438 ≤ t ≤ -432
[[226  64]
 [  0   0]]
> <ipython-input-15-c97f846f7b9c>(9)<module>()
-> pdb.set_trace()
(Pdb) c
-432 ≤ t ≤ -426
[[184  52]

BdbQuit: 